In [1]:
import torch
import torchvision
from torchvision import datasets,transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch import optim
from torch.nn import functional as F

$LeNet5$

In [2]:
def main():
    batch_size = 64
    device = torch.device('cuda')
    model = Lenet5().to(device)
    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3)
    iterations = 10
    print(model)
    
    train_loader = datasets.CIFAR10('./datasets/cifar10',True,
        transform = transforms.Compose([
            transforms.Resize((32,32)),
            transforms.ToTensor()
    ]),download = True)
    train_loader = DataLoader(train_loader,batch_size = batch_size,shuffle = True)
    
    test_loader = datasets.CIFAR10('./datasets/cifar10',False,
        transform = transforms.Compose([
            transforms.Resize((32,32)),
            transforms.ToTensor()
    ]),download = True)
    test_loader = DataLoader(test_loader,batch_size = batch_size,shuffle = True)
    
    x,label = iter(train_loader).next()
    print(f"x : {x.shape} label : {label.shape}")
    
    # train
    for epoch in range(iterations):
        model.train()
        for batch_idx,(x,label) in enumerate(train_loader):
            x,label = x.to(device),label.to(device)
            logits = model(x)
            # logist : [batch,10] y : [batch]
            loss = criteon(logits,label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"epoch : {epoch} loss : {loss.item()}")
        
        # test
        model.eval()
        with torch.no_grad():
            total_num = 0
            total_correct = 0
            for x,label in test_loader:
                x,label = x.to(device),label.to(device)

                logits = model(x)

                pred = torch.argmax(logits,dim = 1)

                total_correct += torch.eq(pred,label).float().sum().item()
                total_num += x.size(0)

            acc = total_correct / total_num
            print(f"epoch : {epoch} accuracy :{acc}")


<img src = "./images/l41_lenet.png"  width = 600 />

In [3]:
class Lenet5(nn.Module):
    
    def  __init__(self):
        super(Lenet5,self).__init__()
        # input [batch,3,32,32] -> [batch,16,5,5]
        self.conv_unit = nn.Sequential(
            nn.Conv2d(3,6,kernel_size = 5,stride = 1,padding  = 0),
            nn.MaxPool2d(kernel_size = 2,stride = 2,padding = 0),
            nn.Conv2d(6,16,kernel_size = 5,stride = 1,padding = 0),
            nn.MaxPool2d(kernel_size = 2,stride = 2,padding = 0)
        )
        self.fc_unit = nn.Sequential(
            nn.Linear(16*5*5,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )
    def forward(self,x):
        # [batch,3,32,32]
        batch_size = x.size(0)
        
        # [batch,16,5,5]
        x = self.conv_unit(x)
        
        # [batch,16*5*5]
        x = x.view(batch_size,-1)
        
        # [batch,10]
        logits = self.fc_unit(x)
        
        return logits

In [4]:
%%time
if __name__ == '__main__':
    main()

Lenet5(
  (conv_unit): Sequential(
    (0): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_unit): Sequential(
    (0): Linear(in_features=400, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)
Files already downloaded and verified
Files already downloaded and verified
x : torch.Size([64, 3, 32, 32]) label : torch.Size([64])
epoch : 0 loss : 1.5117416381835938
epoch : 0 accuracy :0.4743
epoch : 1 loss : 1.1233296394348145
epoch : 1 accuracy :0.5271
epoch : 2 loss : 0.9600183963775635
epoch : 2 accuracy :0.5633
epoch : 3 loss : 0.7850662469863892
epoch : 3 accuracy :0.5719
epoch : 4 loss : 1.4367178678512573
epoch : 4 a

$ResNet$

<img src = "./images/l41_resnet.jpg" width = 400 />

In [5]:
class ResBlock(nn.Module):
    def __init__(self,ch_in,ch_out,stride = 1):
        super(ResBlock,self).__init__()
        
        self.conv1 = nn.Conv2d(ch_in,ch_out,kernel_size = 3,stride = stride,padding = 1)
        self.bn1 = nn.BatchNorm2d(ch_out)
        self.conv2 = nn.Conv2d(ch_out,ch_out,kernel_size = 3,stride = 1,padding = 1)
        self.bn2 = nn.BatchNorm2d(ch_out)
        self.extra = nn.Sequential()
        if ch_in != ch_out:
            self.extra = nn.Sequential(
                nn.Conv2d(ch_in,ch_out,kernel_size = 1,stride = stride),
                nn.BatchNorm2d(ch_out)
            )
    
    def forward(self,x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        # short cut
        out = self.extra(x) + out
        
        return out

<img src = "./images/l41_resnet18.png"/>

In [6]:
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18,self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(3,64,kernel_size = 3,stride = 3,padding = 0),
            nn.BatchNorm2d(64)
        )
        
        # 4 blocks
        
        # [batch,64,h,w] -> [batch,128,h,w]
        self.blk1 = ResBlock(64,128,stride = 2)
        
        # [batch,128,h,w] -> [batch,256,h,w]
        self.blk2 = ResBlock(128,256,stride = 2)
        
        # [batch,256,h,w] -> [batch,512,h,w]
        self.blk3 = ResBlock(256,512,stride = 2)
        
        # [batch,512,h,w] -> [batch,1024,h,w]
        self.blk4 = ResBlock(512,512,stride = 2)
        
        self.outlayer = nn.Linear(512 * 1 * 1,10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        
        x = self.blk1(x)
        x = self.blk2(x)        
        x = self.blk3(x)        
        x = self.blk4(x)
        
#         print('after conv:',x.shape)
        # [batch,512,h,w] -> [batch,512,1,1]
        x = F.adaptive_avg_pool2d(x,[1,1])
#         print('after pool:',x.shape)
        x = x.view(x.size(0),-1)
        x = self.outlayer(x)
        
        return x

In [7]:
def main():
    batch_size = 64
    device = torch.device('cuda')
    model = ResNet18().to(device)
    criteon = nn.CrossEntropyLoss().to(device)
    optimizer = optim.Adam(model.parameters(),lr = 1e-3)
    iterations = 10
    print(model)
    
    train_loader = datasets.CIFAR10('./datasets/cifar10',True,
        transform = transforms.Compose([
            transforms.Resize((32,32)),
            transforms.ToTensor()
    ]),download = True)
    train_loader = DataLoader(train_loader,batch_size = batch_size,shuffle = True)
    
    test_loader = datasets.CIFAR10('./datasets/cifar10',False,
        transform = transforms.Compose([
            transforms.Resize((32,32)),
            transforms.ToTensor()
    ]),download = True)
    test_loader = DataLoader(test_loader,batch_size = batch_size,shuffle = True)
    
    x,label = iter(train_loader).next()
    print(f"x : {x.shape} label : {label.shape}")
    
    # train
    for epoch in range(iterations):
        model.train()
        for batch_idx,(x,label) in enumerate(train_loader):
            x,label = x.to(device),label.to(device)
            logits = model(x)
            # logist : [batch,10] y : [batch]
            loss = criteon(logits,label)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        
        print(f"epoch : {epoch} loss : {loss.item()}")
        
        # test
        model.eval()
        with torch.no_grad():
            total_num = 0
            total_correct = 0
            for x,label in test_loader:
                x,label = x.to(device),label.to(device)

                logits = model(x)

                pred = torch.argmax(logits,dim = 1)

                total_correct += torch.eq(pred,label).float().sum().item()
                total_num += x.size(0)

            acc = total_correct / total_num
            print(f"epoch : {epoch} accuracy :{acc}")


In [8]:
%%time
if __name__ == '__main__':
    main()

ResNet18(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(3, 3))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (blk1): ResBlock(
    (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (bn2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (extra): Sequential(
      (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (blk2): ResBlock(
    (conv1): Conv2d(128, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    (bn1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd